In [1]:
import pulp as plp

#Input data to model in terms of nodes and junctions

N=[0,1,2,3,4,5,6,7,8,9,10,11,12]
Q=[0,1]
Sq=[0.189,1.136]           #length of trains
P0=[2,1,8,9,11]            #path of train 0 going from west to east(node reperesentation)
P1=[12,10,9,5,4,3]         #path of train 1 going from east to west(node reperesentation)
P=[P0,P1]
common_nodes_and_trains_on_it=[[9,0,1]]

J=[1,5,8,9]   #junctions
Segments=[(0,1),(1,2),(1,8),(3,4),(4,5),(5,6),(5,9),(7,8),(7,9),(8,9),(9,10),(9,11),(10,12)]
Segment_lengths=[1.4,1.1,0.3,1,0.5,1.5,0.5,1.5,0.7,0.5,1.0,1.5,1.5]   #miles
speedlimit=40
mu = 0
M = 10000000

In [2]:
B1=[]
for u in range(13):
    B1.append(Segment_lengths[u]/speedlimit)
len(B1)
B2_0=[]
for u in range(13):
    B2_0.append(B1[u]+(Sq[0]/speedlimit))
            
    B2_1=[]
for u in range(13):
    B2_1.append(B1[u]+(Sq[1]/speedlimit))
print(len(B2_1))      

B2=[B2_0,B2_1]

13


In [3]:
# Create a LP Minimization problem 

opt_model = plp.LpProblem('Freight Scheduling',plp.LpMinimize)

# Create problem Variables 

ta =    [[plp.LpVariable(f'ta{q}_{n}',cat=plp.LpContinuous,lowBound = 0)
        for n in P[q]]for q in Q]
    
td =    [[plp.LpVariable(f'td{q}_{n}',cat=plp.LpContinuous,lowBound = 0)
        for n in P[q]]for q in Q]


    
X = [plp.LpVariable(f'X{n}_{q1}_{q2}',cat=plp.LpBinary)
    for n,q1,q2 in common_nodes_and_trains_on_it]


    # Objective Function 

opt_model +=  plp.lpSum(ta[q][-1:] for q in Q)   

In [4]:
# Constraints: 

    
for q in Q :
    temp_P=P[q][:-1]
    for i in range(len(temp_P)):
        path=[P[q][i],P[q][i+1]]
        path.sort()
        path=tuple(path)
        opt_model += ta[q][i+1]-ta[q][i] >= B1[Segments.index(path)]
        
for q in Q :
    temp_P=P[q][:-1]
    for i in range(len(temp_P)):
        path=[P[q][i],P[q][i+1]]
        path.sort()
        path=tuple(path)
        opt_model += td[q][i]-ta[q][i+1] >= B2[q][Segments.index(path)]-B1[Segments.index(path)]

for q in Q:
        path=[P[q][-1],P[q][-2]]
        path.sort()
        path=tuple(path)
        opt_model += td[q][-1]-ta[q][-1] >= B2[q][Segments.index(path)]

for n,q1,q2 in common_nodes_and_trains_on_it:
    opt_model += X[0]*M + ta[q1][P[q1].index(n)]-td[q2][P[q2].index(n)] >= mu
    opt_model += (1-X[0])*M + ta[q2][P[q2].index(n)]-td[q1][P[q1].index(n)] >= mu

In [5]:
status = opt_model.solve() # Solver 
print(plp.LpStatus[status]) # The solution status 

Optimal


In [6]:
# Display the problem 
print(opt_model) 

Freight Scheduling:
MINIMIZE
1*ta0_11 + 1*ta1_3 + 0
SUBJECT TO
_C1: ta0_1 - ta0_2 >= 0.0275

_C2: - ta0_1 + ta0_8 >= 0.0075

_C3: - ta0_8 + ta0_9 >= 0.0125

_C4: ta0_11 - ta0_9 >= 0.0375

_C5: ta1_10 - ta1_12 >= 0.0375

_C6: - ta1_10 + ta1_9 >= 0.025

_C7: ta1_5 - ta1_9 >= 0.0125

_C8: ta1_4 - ta1_5 >= 0.0125

_C9: ta1_3 - ta1_4 >= 0.025

_C10: - ta0_1 + td0_2 >= 0.004725

_C11: - ta0_8 + td0_1 >= 0.004725

_C12: - ta0_9 + td0_8 >= 0.004725

_C13: - ta0_11 + td0_9 >= 0.004725

_C14: - ta1_10 + td1_12 >= 0.0284

_C15: - ta1_9 + td1_10 >= 0.0284

_C16: - ta1_5 + td1_9 >= 0.0284

_C17: - ta1_4 + td1_5 >= 0.0284

_C18: - ta1_3 + td1_4 >= 0.0284

_C19: - ta0_11 + td0_11 >= 0.042225

_C20: - ta1_3 + td1_3 >= 0.0534

_C21: 10000000 X9_0_1 + ta0_9 - td1_9 >= 0

_C22: - 10000000 X9_0_1 + ta1_9 - td0_9 >= -10000000

VARIABLES
0 <= X9_0_1 <= 1 Integer
ta0_1 Continuous
ta0_11 Continuous
ta0_2 Continuous
ta0_8 Continuous
ta0_9 Continuous
ta1_10 Continuous
ta1_12 Continuous
ta1_3 Continuous
ta1_4 Co

In [7]:


print(X[0],plp.value(X[0]))
print("train 1 goes first through junction 9 then train 0")

                
for q in Q:
    for n in range(len(P[q])):
        print(ta[q][n],plp.value(ta[q][n]),"in hours")
        print(td[q][n],plp.value(td[q][n]),"in hours")

X9_0_1 0.0
train 1 goes first through junction 9 then train 0
ta0_2 0.0 in hours
td0_2 0.032225 in hours
ta0_1 0.0275 in hours
td0_1 0.039725 in hours
ta0_8 0.035 in hours
td0_8 0.108125 in hours
ta0_9 0.1034 in hours
td0_9 0.145625 in hours
ta0_11 0.1409 in hours
td0_11 0.183125 in hours
ta1_12 0.0 in hours
td1_12 0.0659 in hours
ta1_10 0.0375 in hours
td1_10 0.0909 in hours
ta1_9 0.0625 in hours
td1_9 0.1034 in hours
ta1_5 0.075 in hours
td1_5 0.1159 in hours
ta1_4 0.0875 in hours
td1_4 0.1409 in hours
ta1_3 0.1125 in hours
td1_3 0.1659 in hours
